In [134]:
import yfinance as yf
import pandas as pd

Skidanje cijena pomoću yfinance paketa:

In [135]:
tickers = ["SPY", "AGG", "GC=F", "VNQ"]

prices = yf.download(tickers, start="2012-01-01", end="2023-12-31")["Adj Close"]

prices

[*********************100%***********************]  4 of 4 completed


Ticker,AGG,GC=F,SPY,VNQ
Date,,,,
2012-01-03 00:00:00+00:00,79.194756,1599.699951,101.091934,35.285412
2012-01-04 00:00:00+00:00,79.086754,1611.900024,101.250511,34.682125
2012-01-05 00:00:00+00:00,79.194756,1619.400024,101.520119,35.007912
2012-01-06 00:00:00+00:00,79.259544,1616.099976,101.258461,34.893284
2012-01-09 00:00:00+00:00,79.194756,1607.500000,101.504242,34.772629
...,...,...,...,...
2023-12-22 00:00:00+00:00,95.843819,2057.100098,469.225250,85.181526
2023-12-26 00:00:00+00:00,96.037766,2058.199951,471.206573,85.812798
2023-12-27 00:00:00+00:00,96.648743,2081.899902,472.058533,86.230400


Čitanje i parsiranje csv dokumenta koji sadrži težine svake klase imovine, ispis težina klasa po godinama

In [136]:
csvfile = pd.read_csv("asset_class_marketcaps.csv", skiprows=1)
marketcap_years = [str(year) for year in range(2012, 2024)]
weights_years = [str(year) + ".1" for year in range(2012, 2024)]

marketcap_df = csvfile[["Asset class"] + marketcap_years]

weights_df = csvfile[["Asset class"] + weights_years]
weights_df.columns = ["Asset class"] + marketcap_years

weights_df = weights_df.set_index("Asset class")

weights_df = weights_df.apply(lambda x: x.str.rstrip("%").astype(float) / 100)

weights_df.style.format("{:,.2%}")

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Asset class,,,,,,,,,,,,
Equity,35.06%,40.81%,40.62%,40.64%,40.67%,42.67%,40.41%,42.43%,42.44%,46.09%,45.04%,46.72%
Fixed income,48.21%,46.00%,46.19%,46.63%,46.53%,44.80%,46.90%,44.99%,45.52%,42.45%,42.95%,41.62%
Commodity,12.27%,8.30%,7.74%,7.10%,7.33%,7.15%,7.29%,7.33%,7.93%,6.92%,8.04%,7.93%
Real estate,4.47%,4.89%,5.46%,5.63%,5.46%,5.38%,5.41%,5.25%,4.12%,4.54%,3.97%,3.73%


Pretvaranje cijena tickera u postotne povrate (razlika u cijeni iz dana u dan)

In [137]:
change = prices.pct_change(1, fill_method=None).dropna()
change

Ticker,AGG,GC=F,SPY,VNQ
Date,,,,
2012-01-04 00:00:00+00:00,-0.001364,0.007626,0.001569,-0.017097
2012-01-05 00:00:00+00:00,0.001366,0.004653,0.002663,0.009394
2012-01-06 00:00:00+00:00,0.000818,-0.002038,-0.002577,-0.003274
2012-01-09 00:00:00+00:00,-0.000817,-0.005321,0.002427,-0.003458
2012-01-10 00:00:00+00:00,-0.000273,0.014619,0.008670,0.011103
...,...,...,...,...
2023-12-22 00:00:00+00:00,-0.001314,0.008827,0.002010,0.003891
2023-12-26 00:00:00+00:00,0.002024,0.000535,0.004223,0.007411
2023-12-27 00:00:00+00:00,0.006362,0.011515,0.001808,0.004866


Pretvaranje dnevnih povrata u godišnje povrate radi konteksta

In [138]:
yearly_change = change.groupby(change.index.year).apply(lambda x: (1 + x).prod() - 1)
yearly_change.index.name = "Year"

yearly_change.style.format("{:,.2%}")

Ticker,AGG,GC=F,SPY,VNQ
Year,,,,
2012,3.96%,4.69%,14.17%,16.65%
2013,-1.98%,-28.24%,32.31%,2.30%
2014,6.00%,-1.50%,13.46%,30.40%
2015,0.48%,-10.44%,1.23%,2.43%
2016,3.22%,12.03%,13.01%,5.80%
2017,3.55%,13.59%,21.71%,4.90%
2018,0.68%,-0.96%,-2.94%,-4.05%
2019,8.46%,18.87%,31.22%,28.91%
2020,7.48%,24.59%,18.33%,-4.61%


Računanje težinskih godišnjih povrata svakog tickera u indeksu (koliko je koja klasa pridonjelja godišnjem povratu indeksa)

In [139]:
asset_class_to_ticker_map = {
    "Equity": "SPY",
    "Fixed income": "AGG",
    "Commodity": "GC=F",
    "Real estate": "VNQ",
}

weights_df_copy = weights_df.copy()

weights_df_copy.rename(index=asset_class_to_ticker_map, inplace=True)
weights_df_copy = weights_df_copy.transpose()
weights_df_copy.index = weights_df_copy.index.astype("int64")

weighted_returns = weights_df_copy.multiply(yearly_change)

weighted_returns.style.format("{:,.2%}")

,AGG,GC=F,SPY,VNQ
2012,1.91%,0.58%,4.97%,0.74%
2013,-0.91%,-2.34%,13.18%,0.11%
2014,2.77%,-0.12%,5.47%,1.66%
2015,0.22%,-0.74%,0.50%,0.14%
2016,1.50%,0.88%,5.29%,0.32%
2017,1.59%,0.97%,9.26%,0.26%
2018,0.32%,-0.07%,-1.19%,-0.22%
2019,3.80%,1.38%,13.25%,1.52%
2020,3.40%,1.95%,7.78%,-0.19%
2021,-0.75%,-0.24%,13.24%,1.84%


Godišnji povrati indeksa dobiveni sumiranjem stupaca

In [140]:
index_returns = weighted_returns.sum(axis=1).to_frame()

index_returns.index.name = "Year"
index_returns.columns = ["Index Returns"]

index_returns.style.format("{:,.2%}")

,Index Returns
Year,
2012,8.20%
2013,10.04%
2014,9.78%
2015,0.12%
2016,7.99%
2017,12.09%
2018,-1.16%
2019,19.95%
2020,12.94%


Računamo betu svakog tickera prema formuli:

$$
\beta = \frac{\mathrm{Cov}(R_{\mathrm{stock}}, R_{\mathrm{market}})}{\mathrm{Var}(R_{\mathrm{market}})}
$$

gdje su:

- $R_{stock}$: Povrat dionice
- $R_{market}$: Povrat indeksa koji smo stvorili
- $\mathrm{Cov}(R_{stock}, R_{market})$: Kovarijanca između povrata dionice i povrata indeksa
- $\mathrm{Var}(R_{market})$: Varijanca povrata indeksa

In [141]:
daily_weights_df = change.index.to_frame()

daily_weights_df["Year"] = daily_weights_df["Date"].dt.year
daily_weights_df = daily_weights_df.merge(weights_df_copy, left_on="Year", right_index=True).set_index("Date")

daily_weights_df = daily_weights_df.drop(columns="Year")

daily_index_change = change.multiply(daily_weights_df).sum(axis=1)

beta_AGG = change["AGG"].cov(daily_index_change) / daily_index_change.var()
beta_SPY = change["SPY"].cov(daily_index_change) / daily_index_change.var()
beta_GC = change["GC=F"].cov(daily_index_change) / daily_index_change.var()
beta_VNQ = change["VNQ"].cov(daily_index_change) / daily_index_change.var()

print("Time period (inclusive): 2012 - 2023")
print(f"Beta AGG: {beta_AGG}")
print(f"Beta SPY: {beta_SPY}")
print(f"Beta GC=F: {beta_GC}")
print(f"Beta VNQ: {beta_VNQ}")

Time period (inclusive): 2012 - 2023
Beta AGG: 0.2108640781248857
Beta SPY: 1.8433056910783463
Beta GC=F: 0.46763904781134413
Beta VNQ: 1.8555039172882282


Račun bete na godišnjoj razini povrata dionice i indeksa:

In [142]:
def yearly_betas(change= change, daily_index_change = daily_index_change, tickers = tickers, start_year = 2012, end_year= 2024):
    yearly_betas = pd.DataFrame(columns=['Year', 'Ticker', 'Beta'])
    
    temp1 = change.copy()
    temp2 = daily_index_change.copy()
    
    for ticker in tickers:
        for year in range(start_year, end_year + 1):
            ticker_changes = temp1[ticker].loc[temp1.index.year == year]
            index_changes = temp2.loc[temp2.index.year == year]
            beta = ticker_changes.cov(index_changes) / index_changes.var()
            yearly_betas = pd.concat(
                [yearly_betas, pd.DataFrame({'Year': [year], 'Ticker': [f"β ({ticker})"], 'Beta': [beta]})],
                ignore_index=True
            )
    
    yearly_betas = yearly_betas.pivot(index="Year", columns="Ticker", values="Beta")
    
    return yearly_betas[:-1]


yearly_betas = yearly_betas()

yearly_betas

/tmp/ipykernel_1662/2317818422.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  yearly_betas = pd.concat(


Ticker,β (AGG),β (GC=F),β (SPY),β (VNQ)
Year,,,,
2012,-0.085622,1.801319,2.120997,1.714424
2013,0.261829,1.763079,1.560220,1.973365
2014,0.008900,0.195935,2.195728,1.626722
2015,0.048091,0.411757,2.062418,1.956906
2016,0.119417,-0.050645,2.033428,2.218905
2017,0.263802,0.717624,1.696749,1.979623
2018,0.021482,0.171447,2.228639,1.420206
2019,0.034877,0.105000,2.128346,1.401052
2020,0.184774,0.395549,1.874214,2.162808


Backtesting po povijesnim podacima koristeći CAPM, prozor od 3 godine.

In [143]:
Rf_rate = yf.download("^IRX", start="2012-01-01", end="2023-12-31")["Adj Close"].dropna()
Rf_rate

[*********************100%***********************]  1 of 1 completed


Ticker,^IRX
Date,
2012-01-03 00:00:00+00:00,0.005
2012-01-04 00:00:00+00:00,0.010
2012-01-05 00:00:00+00:00,0.010
2012-01-06 00:00:00+00:00,0.015
2012-01-09 00:00:00+00:00,0.005
...,...
2023-12-22 00:00:00+00:00,5.208
2023-12-26 00:00:00+00:00,5.203
2023-12-27 00:00:00+00:00,5.235


In [144]:
def rolling_beta(start_year, end_year, window, change=change, daily_index_change=daily_index_change, tickers=tickers):
    window_betas = pd.DataFrame(columns=['Window', 'Ticker', 'Beta'])
    
    temp1 = change.copy()
    temp2 = daily_index_change.copy()
    
    for ticker in tickers:
        window_years = range(start_year, end_year)    
        ticker_changes = temp1[ticker][temp1.index.year.isin(window_years)]
        index_changes = temp2[temp2.index.year.isin(window_years)]
        
        beta = ticker_changes.cov(index_changes) / index_changes.var()
    
        window_betas = pd.concat(
            [window_betas, pd.DataFrame({'Window': [end_year], 'Ticker': [ticker], 'Beta': [beta]})],
            ignore_index=True
        )
    
    window_betas = window_betas.pivot(index="Window", columns="Ticker", values="Beta")
    return window_betas

In [ ]:
# yearly_rf_rate = Rf_rate.groupby(Rf_rate.index.year).median()
# yearly_rf_rate.index.name = "Year"

window = 3

for i in range(2014, 2024, window):
    start_year = i - window + 1
    end_year = i + 1
    window_changes = yearly_change.loc[start_year:end_year-1].sum() # promjena tickera na vremenskom prozoru
    window_betas = rolling_beta(start_year=start_year, end_year=end_year,window=window) # beta izracunana na vremenskom prozoru
    window_rf_rate = Rf_rate.loc[Rf_rate.index.year == end_year-1].iloc[-1] # zadnja vrijednost povrata trezoraca u godini prije one koju predvidamo

    

Ticker
^IRX    0.037
Name: 2014-12-31 00:00:00+00:00, dtype: float64
Ticker
^IRX    1.355
Name: 2017-12-29 00:00:00+00:00, dtype: float64
Ticker
^IRX    0.065
Name: 2020-12-31 00:00:00+00:00, dtype: float64
Ticker
^IRX    5.18
Name: 2023-12-29 00:00:00+00:00, dtype: float64


/tmp/ipykernel_1662/341048004.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  window_betas = pd.concat(
/tmp/ipykernel_1662/341048004.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  window_betas = pd.concat(
/tmp/ipykernel_1662/341048004.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude t